In [1]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score

from sklearn import preprocessing

import pickle
import numpy as np

## Preparing the data

In [2]:
data_path ="/content/drive/MyDrive/Aimtech/"
# data_path ="./"

In [4]:
with open(data_path+"clean_data.pickle", "rb") as f:
    (X,y) = pickle.load(f)

In [5]:
# Splitting the data into train and validation sets with ratio 80 : 20 
X_train, X_test, y_train, y_test = train_test_split(X.flatten(),y, test_size=0.2,stratify=y,random_state=0)
print(X_train.shape)
print(X_test.shape)

(133142,)
(33286,)


In [6]:
# X.flatten()

In [43]:
# vectorizer = TfidfVectorizer(lowercase=False)
# X_test = vectorizer.fit_transform(X_train.flatten() )

In [44]:
# TF-IDF transformation then SVM classifier

pipe_SVM= Pipeline([('tfidf', TfidfVectorizer(lowercase=False)),
               ('clf',svm.SVC(kernel='linear', C=1)),
              ])
pipe_SVM.fit(X_train, y_train)
y_pred_SVM= pipe_SVM.predict(X_test)

#Evaluation
print('\033[1m{:10s}\033[0m'.format('TF-IDF_SVM_model'))
print("Report : \n",classification_report(y_test, y_pred_SVM))
print('\033[1m{:10s}\033[0m'.format("Accuracy:"),accuracy_score(y_test, y_pred_SVM))

TF-IDF_SVM_model
Report :                precision    recall  f1-score   support

          AE       0.32      0.37      0.34      1849
          BH       0.28      0.33      0.30      1849
          DZ       0.53      0.54      0.54      1849
          EG       0.59      0.67      0.63      1849
          IQ       0.55      0.58      0.57      1849
          JO       0.34      0.33      0.34      1849
          KW       0.38      0.36      0.37      1849
          LB       0.56      0.56      0.56      1849
          LY       0.56      0.52      0.54      1849
          MA       0.73      0.62      0.67      1850
          OM       0.33      0.39      0.35      1850
          PL       0.38      0.35      0.36      1849
          QA       0.40      0.38      0.39      1850
          SA       0.31      0.36      0.33      1849
          SD       0.69      0.61      0.64      1850
          SY       0.46      0.40      0.43      1849
          TN       0.66      0.55      0.60      1849


# Deep model

In [59]:
nClasses=len(np.unique(y))
nClasses

18

In [ ]:

le = preprocessing.LabelEncoder()

y_train_le= le.fit_transform(y_train)
y_test_le= le.transform(y_test)

y_train_k = tf.keras.utils.to_categorical(y_train_le, num_classes=nClasses)
y_test_k = tf.keras.utils.to_categorical(y_test_le, num_classes=nClasses)

In [41]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

In [43]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1 

In [44]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [74]:
from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import Adam

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Dropout(0.2))

model.add(layers.LSTM(units=32, return_sequences=True, return_state=False))
# model.add(layers.LSTM(units=16, return_sequences=True, return_state=False))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))

# model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(nClasses, activation='sigmoid'))

opt = Adam(lr = 0.001)
model.compile(optimizer=opt,
              loss=losses.categorical_crossentropy,
              metrics=['accuracy'])



/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [75]:
batchSize = 32
nEpochs = 32

history = model.fit(X_train, y_train_k,
                    epochs=nEpochs,
                    validation_data=(X_test, y_test_k),
                    batch_size=batchSize )

 508/4161 [==>...........................] - ETA: 1:32 - loss: 1.3619 - accuracy: 0.5643

KeyboardInterrupt: ignored

In [66]:
loss, accuracy = model.evaluate(X_train, y_train_k, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test_k, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.6462
Testing Accuracy:  0.3592


In [ ]:
model.save_weights(os.path.join(data_path, 'weights.h5'))